In [1]:
import subprocess
import time

def measure_time(func, *args, **kwargs):
    start = time.perf_counter()
    result = func(*args, **kwargs)
    end = time.perf_counter()
    return result, end - start

def run_command(cmd: str, capture_output: bool = True, timeout: int = None):
    result = subprocess.run(
        cmd,
        shell=True,
        capture_output=capture_output,
        text=True,
        timeout=timeout,
        check=True
    )
    return result

measure_time, run_command

(<function __main__.measure_time(func, *args, **kwargs)>,
 <function __main__.run_command(cmd: str, capture_output: bool = True, timeout: int = None)>)

In [2]:
try:
    # 1. Buat direktori /user/raid/raw
    print("Membuat direktori HDFS /user/raid/raw...")
    result, duration = measure_time(
        run_command,
        "docker exec namenode hdfs dfs -mkdir -p /user/raid/raw"
    )
    print(f"✅ Berhasil dalam {duration:.2f} detik")

    # 2. Set permission ke 777 (read/write/execute untuk semua)
    print("Mengatur permission HDFS ke 777...")
    result, duration = measure_time(
        run_command,
        "docker exec namenode hdfs dfs -chmod -R 777 /user/raid"
    )
    print(f"✅ Berhasil dalam {duration:.2f} detik")

    # 3. Verifikasi
    print("Verifikasi struktur direktori...")
    result, duration = measure_time(
        run_command,
        "docker exec namenode hdfs dfs -ls /user/raid"
    )
    print("Output:\n", result.stdout)

except subprocess.CalledProcessError as e:
    print("❌ Error saat menjalankan perintah:")
    print("Stderr:", e.stderr)
    print("Return code:", e.returncode)
except subprocess.TimeoutExpired:
    print("❌ Timeout saat menjalankan perintah")
except Exception as e:
    print("❌ Error tak terduga:", str(e))

Membuat direktori HDFS /user/raid/raw...
✅ Berhasil dalam 5.90 detik
Mengatur permission HDFS ke 777...
✅ Berhasil dalam 1.30 detik
Verifikasi struktur direktori...
Output:
 Found 1 items
drwxrwxrwx   - root supergroup          0 2025-10-20 07:02 /user/raid/raw

